<a href="https://colab.research.google.com/github/davidzhangsan/cis5450finalproject/blob/main/CIS5450FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import json
import os

from google.colab import files
files.upload()

os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [22]:
!pip install kaggle


In [23]:
!kaggle datasets download -d davidcariboo/player-scores
!unzip -o player-scores.zip -d player_scores_data


Dataset URL: https://www.kaggle.com/datasets/davidcariboo/player-scores
License(s): CC0-1.0
player-scores.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  player-scores.zip
  inflating: player_scores_data/appearances.csv  
  inflating: player_scores_data/club_games.csv  
  inflating: player_scores_data/clubs.csv  
  inflating: player_scores_data/competitions.csv  
  inflating: player_scores_data/game_events.csv  
  inflating: player_scores_data/game_lineups.csv  
  inflating: player_scores_data/games.csv  
  inflating: player_scores_data/player_valuations.csv  
  inflating: player_scores_data/players.csv  
  inflating: player_scores_data/transfers.csv  


In [24]:
# Core libs
import os
from pathlib import Path

import numpy as np
import pandas as pd

# Modeling + preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Try XGBoost
try:
    from xgboost import XGBRegressor
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False
    print("Warning: xgboost not installed. Install via `pip install xgboost` if you want to use it.")

pd.set_option("display.max_columns", 100)

# Project config
LEAGUE_CODE = "GB1"   # Premier League domestic competition id
SEASON = 2022         # 2022 season in `games.season` (e.g. 2022 for 22/23)
REFERENCE_DATE = pd.to_datetime(f"{SEASON+1}-06-30")  # for age calculation


In [25]:
DATA_DIR = Path("player_scores_data")

print("Data directory contents:")
print(list(DATA_DIR.iterdir()))


Data directory contents:
[PosixPath('player_scores_data/clubs.csv'), PosixPath('player_scores_data/game_events.csv'), PosixPath('player_scores_data/games.csv'), PosixPath('player_scores_data/appearances.csv'), PosixPath('player_scores_data/player_valuations.csv'), PosixPath('player_scores_data/competitions.csv'), PosixPath('player_scores_data/game_lineups.csv'), PosixPath('player_scores_data/players.csv'), PosixPath('player_scores_data/club_games.csv'), PosixPath('player_scores_data/transfers.csv')]


In [26]:
players = pd.read_csv(DATA_DIR / "players.csv")
appearances = pd.read_csv(DATA_DIR / "appearances.csv")
games = pd.read_csv(DATA_DIR / "games.csv")
competitions = pd.read_csv(DATA_DIR / "competitions.csv")

print("players:", players.shape)
print("appearances:", appearances.shape)
print("games:", games.shape)
print("competitions:", competitions.shape)

players.head()


players: (32601, 23)
appearances: (1706806, 13)
games: (74026, 23)
competitions: (44, 11)


,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,1978-06-09 00:00:00,Centre-Forward,Attack,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,1980-08-06 00:00:00,Goalkeeper,Goalkeeper,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,1981-01-30 00:00:00,Centre-Forward,Attack,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,1978-05-08 00:00:00,Centre-Back,Defender,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,1981-03-18 00:00:00,Goalkeeper,Goalkeeper,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0


In [27]:
print("Players columns:\n", players.columns.tolist(), "\n")
print("Appearances columns:\n", appearances.columns.tolist(), "\n")
print("Games columns:\n", games.columns.tolist(), "\n")
print("Competitions columns:\n", competitions.columns.tolist(), "\n")

# Look at competitions to confirm LEAGUE_CODE
competitions[['competition_id', 'competition_code', 'name', 'country_name']].drop_duplicates().head(20)

# Dataset size requirement check (rows > 50k for analysis)
print("Total rows (appearances):", len(appearances))
print("Total rows (games):", len(games))
print("Total rows (players):", len(players))


Players columns:
 ['player_id', 'first_name', 'last_name', 'name', 'last_season', 'current_club_id', 'player_code', 'country_of_birth', 'city_of_birth', 'country_of_citizenship', 'date_of_birth', 'sub_position', 'position', 'foot', 'height_in_cm', 'contract_expiration_date', 'agent_name', 'image_url', 'url', 'current_club_domestic_competition_id', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur'] 

Appearances columns:
 ['appearance_id', 'game_id', 'player_id', 'player_club_id', 'player_current_club_id', 'date', 'player_name', 'competition_id', 'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played'] 

Games columns:
 ['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id', 'away_club_id', 'home_club_goals', 'away_club_goals', 'home_club_position', 'away_club_position', 'home_club_manager_name', 'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url', 'home_club_formation', 'away_club_formation', 'home_club_name', 'away_clu

In [28]:
# players date fields
players["date_of_birth"] = pd.to_datetime(players["date_of_birth"], errors="coerce")

# appearances & games use 'date'
appearances["date"] = pd.to_datetime(appearances["date"], errors="coerce")
games["date"] = pd.to_datetime(games["date"], errors="coerce")


In [29]:
# Filter games to chosen league + season
games_league_season = games[
    (games["competition_id"] == LEAGUE_CODE) &
    (games["season"] == SEASON)
].copy()

print("Games in league+season:", games_league_season.shape)

# Filter players whose current club is in that domestic competition
players_league = players[
    players["current_club_domestic_competition_id"] == LEAGUE_CODE
].copy()

print("Players in league:", players_league.shape)

# Join appearances to filtered games (only PL 22/23 matches)
apps_joined = appearances.merge(
    games_league_season[["game_id", "competition_id", "season", "date"]],
    on="game_id",
    how="inner",
    suffixes=("", "_game")
)

print("Appearances in league+season:", apps_joined.shape)
apps_joined.head()


Games in league+season: (380, 23)
Players in league: (2181, 23)
Appearances in league+season: (11345, 16)


,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played,competition_id_game,season,date_game
0,3837814_108354,3837814,108354,873,1003,2022-08-05,Jordan Ayew,GB1,0,0,0,0,90,GB1,2022,2022-08-05
1,3837814_111455,3837814,111455,11,15,2022-08-05,Granit Xhaka,GB1,1,0,0,0,90,GB1,2022,2022-08-05
2,3837814_145988,3837814,145988,873,141,2022-08-05,Wilfried Zaha,GB1,0,0,0,0,90,GB1,2022,2022-08-05
3,3837814_157506,3837814,157506,873,371,2022-08-05,Jeffrey Schlupp,GB1,0,0,0,0,86,GB1,2022,2022-08-05
4,3837814_203853,3837814,203853,11,11,2022-08-05,Oleksandr Zinchenko,GB1,0,0,0,1,83,GB1,2022,2022-08-05


In [30]:
# Stat columns available in appearances
stat_cols = [
    c for c in ["minutes_played", "goals", "assists", "yellow_cards", "red_cards"]
    if c in apps_joined.columns
]
print("Using stat columns:", stat_cols)

agg_dict = {col: "sum" for col in stat_cols}
agg_dict["appearance_id"] = "count"   # number of appearances

season_stats = (
    apps_joined
    .groupby("player_id")
    .agg(agg_dict)
    .rename(columns={"appearance_id": "total_appearances"})
    .reset_index()
)

print("Season stats shape:", season_stats.shape)
season_stats.head()


Using stat columns: ['minutes_played', 'goals', 'assists', 'yellow_cards', 'red_cards']
Season stats shape: (554, 7)


,player_id,minutes_played,goals,assists,yellow_cards,red_cards,total_appearances
0,3333,893,0,1,2,0,31
1,8198,520,1,0,2,0,10
2,14086,2038,1,0,7,0,29
3,16306,2127,4,3,7,2,28
4,17965,2205,0,0,0,0,25


In [31]:
# Compute age at end of season
players_league = players_league.copy()
players_league["age"] = (
    (REFERENCE_DATE - players_league["date_of_birth"]).dt.days / 365.25
)

# Merge season stats into player table
df = players_league.merge(
    season_stats,
    on="player_id",
    how="left"
)

# Fill missing stats (no appearances) with 0
for col in ["total_appearances"] + stat_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Drop rows without market value
df = df[~df["market_value_in_eur"].isna()].copy()

print("Modeling dataframe shape:", df.shape)
df[
    ["player_id", "name", "age", "position", "foot", "height_in_cm",
     "total_appearances"] + stat_cols + ["market_value_in_eur"]
].head(10)


Modeling dataframe shape: (2033, 30)


,player_id,name,age,position,foot,height_in_cm,total_appearances,minutes_played,goals,assists,yellow_cards,red_cards,market_value_in_eur
0,132,Tomas Rosicky,42.735113,Midfield,both,179.0,0.0,0.0,0.0,0.0,0.0,0.0,350000.0
1,488,Gerhard Tremmel,44.618754,Goalkeeper,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,250000.0
2,1397,Michael Owen,43.542779,Attack,both,173.0,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0
3,1573,Thomas Hitzlsperger,41.234771,Midfield,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0
4,2514,Bastian Schweinsteiger,38.910335,Midfield,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0
5,2857,Eldin Jakupovic,38.740589,Goalkeeper,right,191.0,0.0,0.0,0.0,0.0,0.0,0.0,100000.0
6,2998,Robert Huth,38.863792,Defender,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2500000.0
7,3109,Steven Gerrard,43.082820,Midfield,right,183.0,0.0,0.0,0.0,0.0,0.0,0.0,1750000.0
8,3118,Kieron Dyer,44.501027,Midfield,right,170.0,0.0,0.0,0.0,0.0,0.0,0.0,500000.0
9,3120,Richard Wright,45.648186,Goalkeeper,right,190.0,0.0,0.0,0.0,0.0,0.0,0.0,250000.0


In [32]:
# Lean feature set ≈ 10 features: age, height, position, foot, apps + stats
BASE_FEATURES = [
    "age",
    "height_in_cm",
    "position",
    "foot",
    "total_appearances",
] + stat_cols

TARGET_COL = "market_value_in_eur"

df_model = df.dropna(subset=[TARGET_COL] + BASE_FEATURES).copy()

X = df_model[BASE_FEATURES]
y = df_model[TARGET_COL].values

numeric_features = [f for f in BASE_FEATURES if f not in ["position", "foot"]]
categorical_features = ["position", "foot"]

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train size:", X_train.shape, "Test size:", X_test.shape)


Train size: (1483, 10) Test size: (371, 10)


In [35]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def mape(y_true, y_pred):
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0

def eval_regression_model(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)              # <-- no `squared=False`, compute RMSE manually
    r2 = r2_score(y_true, y_pred)
    mape_val = mape(y_true, y_pred)
    print(f"\n=== {name} ===")
    print(f"MAE  : {mae:,.2f} EUR")
    print(f"RMSE : {rmse:,.2f} EUR")
    print(f"R^2  : {r2:,.4f}")
    print(f"MAPE : {mape_val:,.2f}%")
    return {"MAE": mae, "RMSE": rmse, "R2": r2, "MAPE": mape_val}


In [36]:
results = {}

# 1) Ridge baseline (interpretable)
ridge_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", Ridge())
    ]
)

ridge_param_grid = {
    "model__alpha": [0.1, 1.0, 10.0, 50.0, 100.0]
}

ridge_cv = GridSearchCV(
    ridge_pipe,
    ridge_param_grid,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)
ridge_cv.fit(X_train, y_train)
print("Best Ridge params:", ridge_cv.best_params_)

ridge_best = ridge_cv.best_estimator_
y_pred_ridge = ridge_best.predict(X_test)
results["Ridge"] = eval_regression_model("Ridge", y_test, y_pred_ridge)


# 2) Random Forest
rf_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", RandomForestRegressor(
            n_estimators=300,
            random_state=42,
            n_jobs=-1
        ))
    ]
)

rf_param_grid = {
    "model__max_depth": [6, 10, None],
    "model__max_features": ["sqrt", 0.7]
}

rf_cv = GridSearchCV(
    rf_pipe,
    rf_param_grid,
    cv=3,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)
rf_cv.fit(X_train, y_train)
print("Best RF params:", rf_cv.best_params_)

rf_best = rf_cv.best_estimator_
y_pred_rf = rf_best.predict(X_test)
results["RandomForest"] = eval_regression_model("RandomForest", y_test, y_pred_rf)


# 3) XGBoost (if available)
if XGB_AVAILABLE:
    xgb_pipe = Pipeline(
        steps=[
            ("preprocess", preprocessor),
            ("model", XGBRegressor(
                objective="reg:squarederror",
                n_estimators=300,
                learning_rate=0.05,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                n_jobs=-1
            ))
        ]
    )

    xgb_param_grid = {
        "model__max_depth": [3, 6],
        "model__min_child_weight": [1, 5],
    }

    xgb_cv = GridSearchCV(
        xgb_pipe,
        xgb_param_grid,
        cv=3,
        scoring="neg_mean_absolute_error",
        n_jobs=-1
    )
    xgb_cv.fit(X_train, y_train)
    print("Best XGB params:", xgb_cv.best_params_)

    xgb_best = xgb_cv.best_estimator_
    y_pred_xgb = xgb_best.predict(X_test)
    results["XGBoost"] = eval_regression_model("XGBoost", y_test, y_pred_xgb)
else:
    print("Skipping XGBoost – not installed.")


Best Ridge params: {'model__alpha': 100.0}

=== Ridge ===
MAE  : 7,111,805.75 EUR
RMSE : 13,233,350.57 EUR
R^2  : 0.3480
MAPE : 1,532.91%
Best RF params: {'model__max_depth': 10, 'model__max_features': 0.7}

=== RandomForest ===
MAE  : 5,952,965.43 EUR
RMSE : 12,397,066.09 EUR
R^2  : 0.4278
MAPE : 939.33%
Best XGB params: {'model__max_depth': 3, 'model__min_child_weight': 1}

=== XGBoost ===
MAE  : 6,151,395.06 EUR
RMSE : 12,111,341.01 EUR
R^2  : 0.4539
MAPE : 966.77%


In [37]:
# Attach test rows back to df_model for stratification
test_idx = X_test.index
test_df = df_model.loc[test_idx].copy()
test_df["y_true"] = y_test
test_df["y_pred_ridge"] = y_pred_ridge
test_df["y_pred_rf"] = y_pred_rf
if XGB_AVAILABLE:
    test_df["y_pred_xgb"] = y_pred_xgb

def group_mape(df, pred_col, mask, label):
    sub = df[mask]
    if len(sub) == 0:
        print(f"{label}: no samples")
        return
    val = mape(sub["y_true"], sub[pred_col])
    print(f"{label} ({len(sub)} players) MAPE: {val:,.2f}%")

young_mask = test_df["age"] < 23
prime_mask = (test_df["age"] >= 23) & (test_df["age"] <= 30)
veteran_mask = test_df["age"] > 30

for model_name, pred_col in [
    ("Ridge", "y_pred_ridge"),
    ("RandomForest", "y_pred_rf"),
] + ([("XGBoost", "y_pred_xgb")] if XGB_AVAILABLE else []):
    print(f"\n=== {model_name} by age band ===")
    group_mape(test_df, pred_col, young_mask, "Young (<23)")
    group_mape(test_df, pred_col, prime_mask, "Prime (23–30)")
    group_mape(test_df, pred_col, veteran_mask, "Veteran (>30)")



=== Ridge by age band ===
Young (<23) (101 players) MAPE: 1,941.87%
Prime (23–30) (114 players) MAPE: 1,782.54%
Veteran (>30) (156 players) MAPE: 1,085.72%

=== RandomForest by age band ===
Young (<23) (101 players) MAPE: 2,001.18%
Prime (23–30) (114 players) MAPE: 980.48%
Veteran (>30) (156 players) MAPE: 221.77%

=== XGBoost by age band ===
Young (<23) (101 players) MAPE: 1,951.19%
Prime (23–30) (114 players) MAPE: 984.71%
Veteran (>30) (156 players) MAPE: 316.32%


In [38]:
# Fit preprocessor to get feature names
preprocessor.fit(X_train)
X_train_pre = preprocessor.transform(X_train)

rf_simple = RandomForestRegressor(
    n_estimators=300,
    max_depth=rf_cv.best_params_["model__max_depth"],
    max_features=rf_cv.best_params_["model__max_features"],
    random_state=42,
    n_jobs=-1
)
rf_simple.fit(X_train_pre, y_train)

feature_names = preprocessor.get_feature_names_out()
importances = rf_simple.feature_importances_

feat_imp = (
    pd.Series(importances, index=feature_names)
    .sort_values(ascending=False)
)

feat_imp.head(20)


,0
num__age,0.288245
num__minutes_played,0.195129
num__total_appearances,0.148567
num__goals,0.112640
num__height_in_cm,0.085073
num__assists,0.076753
num__yellow_cards,0.029582
cat__position_Midfield,0.018771
cat__foot_left,0.011048
cat__foot_right,0.009715
